In [0]:
dbutils.fs.unmount("/mnt/sourcemount")
dbutils.fs.unmount("/mnt/destinationmount")
dbutils.fs.mount(
     source = "wasbs://sourcecontainer@storageaccount908.blob.core.windows.net",
     mount_point = "/mnt/sourcemount",
     extra_configs = {"fs.azure.account.key.storageaccount908.blob.core.windows.net": "+6PBl7VzPeSPrIC2F3oQQpyBUZhHrsiO3UerhQPSvVkRTDxZ2ecXCMcbDdJG53Q+jFT0851OEgR3+ASti17/3w=="})

In [0]:
dbutils.fs.mount(
     source = "wasbs://destinationcontainer@storageaccount908.blob.core.windows.net",
     mount_point = "/mnt/destinationmount",
     extra_configs = {"fs.azure.account.key.storageaccount908.blob.core.windows.net": "+6PBl7VzPeSPrIC2F3oQQpyBUZhHrsiO3UerhQPSvVkRTDxZ2ecXCMcbDdJG53Q+jFT0851OEgR3+ASti17/3w=="})

In [0]:
df = spark.read.csv('/mnt/sourcemount/PROJECT 2 CSV DATA.csv',inferSchema=True, header =True)
df.show()

In [0]:
from pyspark.sql import functions as F
df = df.withColumn('is_police',\
     F.when(\
     F.lower(\
     F.col('local_site_name')).contains('police'),\
     F.lit(1)).\
                    otherwise(F.lit(0)))
df.select('is_police', 'local_site_name').show()

In [0]:
parameter_list = ['Police', 'Fort' , 'Lab']
df = df.withColumn('rating',\
     F.when(\
     F.col('local_site_name').rlike('|'.join(parameter_list)),\
     F.lit('High Rating')).\
     otherwise(F.lit('Low Rating')))
df.select('rating', 'local_site_name').show()

In [0]:
df = df.withColumn('rating', F.when(F.lower(F.col('local_site_name')).contains('police'), F.lit('High Rating'))\
                              .when(F.lower(F.col('local_site_name')).contains('fort'), F.lit('High Rating'))\
                              .when(F.lower(F.col('local_site_name')).contains('lab'), F.lit('High Rating'))\
                              .otherwise(F.lit('Low Rating')))
df.select('rating', 'local_site_name').show()

In [0]:
df = df.withColumn('address', F.trim(F.col('address')))
df.show()

In [0]:
filtered_data = df.filter((F.col('pollutant_standard').isNotNull())) # filter out nulls
filtered_data.count()

In [0]:
filtered_data = df.filter((F.col('event_type').isNotNull()) | (F.col('site_num').isNotNull())) # filter out nulls
filtered_data.count()

In [0]:
filtered_data = df.na.drop(how = 'all') # filter out nulls
filtered_data.show()


In [0]:
filtered_data.write.csv('/mnt/destinationmount/output.csv')